# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [1]:
# Creamos el directorio para los archivos de salida

# Si se corre en Colab, correr las lines de abajo.
'''
!mkdir /content/output
!git clone https://github.com/jingibre/DL-Aug-2020-Trabajo-Final-CV-MJin.git
%cd DL-Aug-2020-Trabajo-Final-CV-MJin
'''

'\n!mkdir /content/output\n!git clone https://github.com/jingibre/DL-Aug-2020-Trabajo-Final-CV-MJin.git\n%cd DL-Aug-2020-Trabajo-Final-CV-MJin\n'

In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_gpu_available()
!python --version


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13757058677128884965
]
2.4.1
Num GPUs Available:  0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Python 3.8.3


In [3]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [4]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

# Descomentar esta linea para usar las imagenes por defecto originales

base_image_path = Path("original source images/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("original source images/La_noche_estrellada1.jpg")

# Descomentar esta linea para obtener la combinación de El Colo y La pintua de Van Gogh
'''
base_image_path = Path("source images/colo_3.jpeg")
style_reference_image_path = Path("source images/van_3.jpg")
'''

'''
# Descomentar esta linea para obtener la combinación de El Verdadero Van Gogh y La pintua de Van Gogh
base_image_path = Path("source images/real_van.jpg")
style_reference_image_path = Path("source images/van_2.jpg")
'''

result_prefix = Path("content/output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:
- total_variation_weight: Este parametro pesa/pondera el valor que se entrega a la loss de suavizado de la imagen (explicada en la respuesta 5) frente a las losses de estilo y contenido. Esta loss no se encuentra definida en el paper.
- style_weight & content_weight: Son los pesos/ponderacion del estilo y contenido respectivamente. Haciendo referencia al trabajo, *content_weight* sería $\alpha$ y *style_weight* sería $\beta$. Estos parametros ponderan las losses regulando asi la importancia que se le da a cada caracteristica a la hora de generar la nueva imagen (que combina estilo y contenido).


In [5]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [6]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta: 
- La funcion detallada en la siguiente celda prepara la imagen para ser introducida en la red CNN VGG19. En principio la carga y luego la convierte en un array 3D (primeras dos lineas). 
- Luego, sabiendo que el input de la VGG es (batch_size, image_width, image_height, image_channels), se le debe agregar la dimension de batch_size, y esa justamente es la linea de expand. 
- Por último, el paso de pre_process, es un step requerido por la arquitectura de la red. En el caso de VGG19, se convierte a la imagen de RGB en BGR, donde se centra cada color de la imagen, con respecto a la media del dataset de ImageNet.

In [7]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta: 
- Sabiendo que lo que hace el pre_process es restarle/centrar todos los canales RGB con respecto de las medias de ImageNet, también sabemos que el resultado de la imagen que generemos estará centrada en esas medias. Por esa razón, para volver a tener una imagen "real" o RGB, lo que hay que hacer, es simplemente de-centrar/sumarle el el valor de las medias a cada canal de la imagen.


In [8]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [9]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [10]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [11]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [12]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?
> La matriz de Gram contiene/computa la correlación de _features_ que existe entre filtros de una misma capa, esto actua como una definición de _estilo_. Esta matriz es la clave para entregarle a la imagen generada el estilo deseado (la generada debe tener la misma matriz de Gram que aquella que se desea copiar el estilo). A nivel modelo, esta matriz es el la que se utiliza en la Loss de estilo.








- ¿Por qué se permutan las dimensiones de x?
> Se permutan las dimensiones, para llevar la dimension de canales [2] (RGB, o BGR en realidad siendo VGG) a la primer dimension, y así, al aplicar el flatten me genere 3 tensores, uno para cada canal.

In [13]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:
- La _style_loss_ mide la diferencia que existe entre el estilo (o matriz de Gram) de la imagen semilla vs el estilo de la imagen generada (que combina estilo de la semilla y contenido de otra).
- La _content_loss_ evalua la diferencia entre features de una layer para la imagen de contenido semilla (base) y aquellos de la imagen generada (combination). En otras palabras, evalua la diferencia de contenido que hay entre la imagen de referencia y la generada.
- La _total_variation_loss mide la diferencia que hay entre pixeles adyacentes, esta loss al ser minimizada, actua entonces, como una suavizadora de cambios en la imagen / ruido.

In [14]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [15]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [16]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [17]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [18]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

- _def eval_loss_and_grads(x)_: Esta función devuelve el valor de los gradientes y de la Loss para una determinada imagen combinada generada.  
- _class Evaluator_: Está clase/objeto se encarga justamente de evaluar/contener las losses y gradientes durante el proceso de optimizado. Hace uso, claramente, la función definida arriba.
- _fmin_l_bfgs_b_: Esta función es la que se encarga de minimizar la Loss que fue definida arriba. Este algoritmo de minimización difiere de los utilizados en el curso hasta ahora ya que es de _segundo orden_. Esto quiere decir que usan la matriz Heassiana o alguna aproximación de la misma para tomar en cuenta la curvatura (no solamente la pendiente) de la función objetivo. L-BFGS es un metodo de segundo orden y tiene la ventaja además de utilizar menos memoria que otros metodos de optimización de este estilo.

La implementación presentada en el código presenta algunas diferencias con el paper:


1.   El termino de la loss asociado a total_variation no se encuentra descrito en el paper donde solamente se asigna un termino para estilo y otro para contenido. No obstante, su adición mejora el resultado del código ya que suaviza satisfactoriamente el ruido en las imagenes.
2.   En el paper indica que cambiar las operaciones de max-pooling for aquellas de average pooling mejora el flujo del gradiente. Sin embargo, en esta implementación estás capas no se ven modificadas en el _model_ dejando así max-pooling en las capas de down sampling.
3.   Por último, la layer elegida para la reconstrucción del contenido difiere levemente de la mencionada en el paper, siendo el bloque 5 el elegido y no el 4 como indica el documento. Esta última diferencia no resulta crucial, y tampoco los autores indicaban que el bloque 4 de la conv 2 fuese el único posible.  De hecho, solo mencionan que la elección de capas menores reconstruyen a la perfección la imagen, y aquellas mas profundas preservan unicamente información de alto orden.






In [19]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [20]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [21]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 13273807000.0
Image saved as content\output\output_at_iteration_0.png
Iteration 0 completed in 87s
Start of iteration 1


KeyboardInterrupt: 

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta: